In [1]:
from xgcm import Grid
import pop_tools
import gcsfs
import fsspec as fs
import numpy as np
import xesmf as xe
import xarray as xr
import random
import matplotlib.pyplot as plt
import warnings
from xgcm import Grid
import importlib
import preprocessing
import os
import xrft
import gcm_filters
import random

warnings.filterwarnings("ignore")

importlib.reload(preprocessing)
from preprocessing import preprocess_data

from gcm_filtering import filter_inputs_dataset
from gcm_filtering import filter_inputs

#import torch
#import torch.nn as nn
#import torch.optim as optim
#import torchvision.transforms as transforms
#from torch.utils.data import DataLoader, Dataset


In [2]:
ds_merged = preprocess_data()
ds_merged

Load Data
Interpolating ocean velocities
Modify units
Load Data
Align in time
Interpolating ocean velocities
Regrid Atmospheric Data
Modify units
Mask nans
Calculate relative wind


<xarray.Dataset> Size: 3TB
Dimensions:       (time: 7305, yt_ocean: 2700, xt_ocean: 3600)
Coordinates:
  * time          (time) object 58kB 0181-01-01 12:00:00 ... 0200-12-31 12:00:00
  * xt_ocean      (xt_ocean) float64 29kB -279.9 -279.8 -279.7 ... 79.85 79.95
  * yt_ocean      (yt_ocean) float64 22kB -81.11 -81.07 -81.02 ... 89.94 89.98
    area_t        (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    dxt           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    dyt           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    geolat_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    geolon_t      (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    ht            (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    kmt           (yt_ocean, xt_ocean) float32 39MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
    wet           (yt_ocean, xt_ocean) float64 78MB dask.array<chunksize=(2700, 3600), meta=np.ndarray>
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    rh_ref        (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 284GB dask.array<chunksize=(3, 2700, 3600), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [40]:
ds_in = ds_merged.drop(['area_t', 'dxt', 'dyt', 'geolat_t', 'geolon_t', 'ht', 'kmt', 'wet'])

print("Filtering model")
ds_temp = ds_in.isel(time=slice(1460,1825)) # increment for each 1 yr
ds_out = filter_inputs_dataset(ds_temp, ['yt_ocean', 'xt_ocean'], filter_scale=50, filter_type='gaussian')

ds_lowRes = ds_out.sel(yt_ocean=slice(-30, -15), xt_ocean=slice(-25, -10)) # South Atlantic
ds_lowRes

Filtering model


<xarray.Dataset> Size: 428MB
Dimensions:       (time: 365, xt_ocean: 150, yt_ocean: 163)
Coordinates:
  * time          (time) object 3kB 0184-12-31 12:00:00 ... 0185-12-30 12:00:00
  * xt_ocean      (xt_ocean) float64 1kB -24.95 -24.85 -24.75 ... -10.15 -10.05
  * yt_ocean      (yt_ocean) float64 1kB -29.98 -29.89 -29.81 ... -15.17 -15.07
Data variables:
    slp           (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    v_ref         (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    u_ref         (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    t_ref         (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    q_ref         (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    wind          (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    rh_ref        (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    surface_temp  (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    u_ocean       (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    v_ocean       (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    u_relative    (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
    v_relative    (time, yt_ocean, xt_ocean) float32 36MB dask.array<chunksize=(1, 163, 150), meta=np.ndarray>
Attributes:
    filter_scale:  50
    filter_type:   gaussian

In [41]:
from distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4, threads_per_worker=2) #default: n_workers=4, threads_per_worker=1
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/panalluri/proxy/35119/status,
Dashboard: /user/panalluri/proxy/35119/status,Workers: 4
Total threads: 8,Total memory: 125.80 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39741,Workers: 4
Dashboard: /user/panalluri/proxy/35119/status,Total threads: 8
Started: Just now,Total memory: 125.80 GiB
Comm: tcp://127.0.0.1:37139,Total threads: 2
Dashboard: /user/panalluri/proxy/38121/status,Memory: 31.45 GiB
Nanny: tcp://127.0.0.1:39407,


In [42]:
# Set number of time samples
time_training = ds_lowRes['time'][::10]
time_len = len(time_training)

# Define the dimensions
xt_len = len(ds_lowRes.xt_ocean) 
yt_len = len(ds_lowRes.yt_ocean) 

# Extract needed data
urel_data = ds_lowRes.u_relative.sel(time=time_training).values
vrel_data = ds_lowRes.v_relative.sel(time=time_training).values
sst_data = ds_lowRes.surface_temp.sel(time=time_training).values
tref_data = ds_lowRes.t_ref.sel(time=time_training).values
qref_data = ds_lowRes.q_ref.sel(time=time_training).values
slp_data = ds_lowRes.slp.sel(time=time_training).values

# Define coordinates
xt_ocean = ds_lowRes.xt_ocean
yt_ocean = ds_lowRes.yt_ocean

# Preallocate the Dataset
ds_training = xr.Dataset(
    {
        'u_rel': (['time', 'yt_ocean', 'xt_ocean'], urel_data),  # Preallocated NaN array
        'v_rel': (['time', 'yt_ocean', 'xt_ocean'], vrel_data),      
        't_ref': (['time', 'yt_ocean', 'xt_ocean'], tref_data),
        'q_ref': (['time', 'yt_ocean', 'xt_ocean'], qref_data),
        'sst': (['time', 'yt_ocean', 'xt_ocean'], sst_data),
        'slp': (['time', 'yt_ocean', 'xt_ocean'], slp_data)
    },
    coords={
        'time': time_training,
        'xt_ocean': xt_ocean,
        'yt_ocean': yt_ocean
    }
)

ds_training

<xarray.Dataset> Size: 43MB
Dimensions:   (time: 37, yt_ocean: 163, xt_ocean: 150)
Coordinates:
  * time      (time) object 296B 0184-12-31 12:00:00 ... 0185-12-26 12:00:00
  * xt_ocean  (xt_ocean) float64 1kB -24.95 -24.85 -24.75 ... -10.15 -10.05
  * yt_ocean  (yt_ocean) float64 1kB -29.98 -29.89 -29.81 ... -15.17 -15.07
Data variables:
    u_rel     (time, yt_ocean, xt_ocean) float64 7MB -0.5207 -0.4796 ... -7.753
    v_rel     (time, yt_ocean, xt_ocean) float64 7MB 1.202 1.183 ... 3.154 3.173
    t_ref     (time, yt_ocean, xt_ocean) float64 7MB 291.6 291.6 ... 294.3 294.3
    q_ref     (time, yt_ocean, xt_ocean) float64 7MB 0.01117 0.01117 ... 0.0128
    sst       (time, yt_ocean, xt_ocean) float64 7MB 293.8 293.8 ... 295.6 295.6
    slp       (time, yt_ocean, xt_ocean) float64 7MB 1.012e+05 ... 1.016e+05

In [43]:
# save to bucket
ds_training.to_zarr("gs://leap-persistent/panalluri/ds_trainingData_southAtlantic_every10TimeSteps_yr5.zarr")